In [1]:
import open3d as o3d
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
img_color = cv2.imread('test_imgs/color_new.png')
img_depth = cv2.imread('test_imgs/depth_new.png', 0)
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

color shape:  (480, 640, 3)
depth shape:  (480, 640)


In [4]:
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[322.282420, 0, 320.818268],[0, 322.282420, 178.779297],[0, 0, 1]]

In [5]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [6]:
pcd = get_pointcloud_from_color_depth(color_image=img_color, depth_image=img_depth, intrinsic=intrinsic)

In [7]:
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd])

PointCloud with 276695 points.
[[-0.00044893 -0.00025017  0.00045098]
 [-0.00044753 -0.00025017  0.00045098]
 [-0.00044613 -0.00025017  0.00045098]
 ...
 [ 0.00022062  0.00021919  0.00023529]
 [ 0.00022135  0.00021919  0.00023529]
 [ 0.00022208  0.00021919  0.00023529]]


In [12]:
mesh = o3d.io.read_triangle_mesh("mesh/jenga_tower_side_xy.stl")
print(mesh)

TriangleMesh with 12 points and 4 triangles.


In [19]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50, origin=[0, 0, 0])

In [20]:
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh, mesh_frame])

In [21]:
pcd = mesh.sample_points_uniformly(number_of_points=5000)
o3d.visualization.draw_geometries([pcd, mesh_frame])